In [33]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error

from tensorflow.keras.optimizers.schedules import ExponentialDecay
from sklearn.ensemble import BaggingRegressor

# Đọc dữ liệu từ CSV vào DataFrame
data = pd.read_csv('/kaggle/input/throughput-prediction/datasetNov1_data_full.csv')

In [34]:
# Chia input và output
X = data.iloc[:, 0:11].values
y = data.iloc[:, 11:13].values
print(X)
print(y)

[[5.0e+00 2.0e+00 1.0e+00 ... 3.4e+02 5.9e+02 1.0e+00]
 [3.0e+00 5.0e+00 6.0e+00 ... 3.8e+02 5.5e+02 1.5e+00]
 [5.0e+00 5.0e+00 4.0e+00 ... 3.5e+02 5.7e+02 5.0e-01]
 ...
 [4.0e+00 4.0e+00 3.0e+00 ... 3.2e+02 5.0e+02 1.5e+00]
 [6.0e+00 3.0e+00 4.0e+00 ... 4.0e+02 5.3e+02 5.0e-01]
 [1.0e+00 4.0e+00 3.0e+00 ... 3.7e+02 5.0e+02 1.5e+00]]
[[0.00000000e+00 6.22222222e-01]
 [1.11239380e-02 8.57025920e-01]
 [0.00000000e+00 3.05555556e-01]
 ...
 [9.76509144e-01 2.27078280e-02]
 [3.01000000e-05 2.77769408e-01]
 [1.55790585e-01 7.45718317e-01]]


In [35]:
# Chuẩn hóa dữ liệu
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
y = scaler.fit_transform(y)




In [36]:
# Xây dựng mô hình mạng học sâu
# Thay đổi kiến trúc mạng
model = Sequential([
    Dense(512, activation='relu', input_shape=(11,)),
   # Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(128, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(2),
])

model.summary()
# Compile mô hình
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 512)               6144      
                                                                 
 dense_25 (Dense)            (None, 128)               65664     
                                                                 
 dense_26 (Dense)            (None, 128)               16512     
                                                                 
 dense_27 (Dense)            (None, 32)                4128      
                                                                 
 dense_28 (Dense)            (None, 16)                528       
                                                                 
 dense_29 (Dense)            (None, 2)                 34        
                                                                 
Total params: 93010 (363.32 KB)
Trainable params: 9301

In [37]:
# Compile the model with learning rate schedule
initial_learning_rate = 0.001
lr_schedule = ExponentialDecay(
    initial_learning_rate, decay_steps=10000, decay_rate=0.9, staircase=True
)

model.compile(optimizer=Adam(learning_rate=lr_schedule), loss='mean_squared_error')


In [38]:
# Chia dữ liệu thành tập huấn luyện và tập xác nhận
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.5, random_state=42)


In [39]:
# Huấn luyện mô hình
history = model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_val, y_val), verbose=1)

Epoch 1/100
651/651 [==============================] - 7s 9ms/step - loss: 0.0200 - val_loss: 0.0117
Epoch 2/100
651/651 [==============================] - 6s 10ms/step - loss: 0.0032 - val_loss: 0.0016
Epoch 3/100
651/651 [==============================] - 5s 7ms/step - loss: 0.0014 - val_loss: 0.0016
Epoch 4/100
651/651 [==============================] - 5s 7ms/step - loss: 0.0010 - val_loss: 6.5170e-04
Epoch 5/100
651/651 [==============================] - 6s 8ms/step - loss: 7.8662e-04 - val_loss: 0.0012
Epoch 6/100
651/651 [==============================] - 4s 7ms/step - loss: 7.5882e-04 - val_loss: 8.5690e-04
Epoch 7/100
651/651 [==============================] - 6s 8ms/step - loss: 7.2746e-04 - val_loss: 5.2615e-04
Epoch 8/100
651/651 [==============================] - 5s 7ms/step - loss: 5.6773e-04 - val_loss: 4.9373e-04
Epoch 9/100
651/651 [==============================] - 5s 7ms/step - loss: 5.2635e-04 - val_loss: 6.0898e-04
Epoch 10/100
651/651 [============================

In [40]:
# Đánh giá hiệu năng trên tập validation bằng RMSE
y_pred = model.predict(X_val)
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print("MSE:", mse)
print("RMSE trên tập validation:", rmse)

2602/2602 [==============================] - 5s 2ms/step
MSE: 2.3599932937325072e-05
RMSE trên tập validation: 0.004857976218274959
